# 探索的可視化と仮説の設定

## 進捗状況
- データの前処理、整形

## ToDO
- (データの整形)
- タグの見方の確認
- 探索的可視化
- 文書全体の処理、可視化

In [50]:
import os
sd_folder = './MultiEnJa/source-document'
mt_folder = './MultiEnJa/MT-PE/en-ja.mt'
ht_folder = './MultiEnJa/human-translation/en-ja.final'

#print(len(os.listdir(mt_folder))) # 46
#print(len(os.listdir(ht_folder))) # 46
#print(os.listdir(ht_folder))
#print(os.listdir(mt_folder) == os.listdir(ht_folder)) # True

for file in os.listdir(mt_folder):
    sd_file = sd_folder + '/' + file
    with open(sd_file, 'r') as f:
        print(f.read())

"""
mt_ids = {}
for mt_file in os.listdir(mt_folder):
    mt_path = os.path.join(mt_folder, mt_file)
    mt_id, _ = os.path.splitext(mt_file)
    #if mt_path.endswith('.txt'):
        #with open(mt_path, 'r') as f:
        #    print(f.read())
"""

Electronic Common Technical Document (eCTD)
The eCTD is the standard format for submitting applications, amendments, supplements, and reports to FDA's Center for Drug Evaluation and Research (CDER) and Center for Biologics Evaluation and Research (CBER).
Important Dates
Reminder: Per eCTD Guidance, NDA, BLA, ANDA, and Commercial IND submissions must include a FDA fillable form (e.g. 356h, 1571, 2252). FDA plans to implement a validation check for the fillable form beginning July 27, 2018. Submissions which fail this validation will be subject to rejection. Please see the eCTD Guidance and the eCTD Validation Criteria (error code 7) for details.
After the dates listed below, eCTD requirements for submissions to CDER and CBER will go into effect and submissions that do not use eCTD will not be filed or received.
May 5, 2017:
New Drug Applications (NDAs), Abbreviated NDAs (ANDAs), and Biologics License Applications (BLAs), must be submitted using eCTD format.
May 5, 2018:
Commercial Inves

"\nmt_ids = {}\nfor mt_file in os.listdir(mt_folder):\n    mt_path = os.path.join(mt_folder, mt_file)\n    mt_id, _ = os.path.splitext(mt_file)\n    #if mt_path.endswith('.txt'):\n        #with open(mt_path, 'r') as f:\n        #    print(f.read())\n"

In [49]:
# e.g., 00000088-B-2-X-9.txt
# 文分割(タイトルや見出しはそのまま)
import spacy
import re

url = r"https?://\S+" # URL
email = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}" # メールアドレス
phone = r"\+?\d{1,4}[-.\s]?\(?\d{1,4}\)?[-.\s]?\d{1,4}[-.\s]?\d{1,4}(?:[-.\s]?\d{1,4})?" # 電話番号

nlp = spacy.load("ja_ginza")
mt_file = mt_folder + '/' + '00000088-B-2-X-9.txt'
mt_sent = []
mt_info = {}
with open(mt_file, 'r') as f:
    for line in f:
        for sent in nlp(line.strip()).sents:
            sent = str(sent)
            sent = re.sub(url, "URL", sent)
            sent = re.sub(email, "MAIL", sent)
            sent = re.sub(phone, "PHONE", sent)
            mt_sent.append(str(sent))
for i, sent in enumerate(mt_sent):
    sent_info = {'sent_id': str(i), 'sent': sent}
    #print(sent)
    for j, token in enumerate(nlp(sent)):
        ja_pos = token.tag_.split('-')
        if len(ja_pos) == 1:
            pos1, pos2, pos3 = ja_pos[0], 'N', 'N'
        elif len(ja_pos) == 2:
            pos1, pos2, pos3 = ja_pos[0], ja_pos[1], 'N'
        else: # len(ja_pos) == 3:
            pos1, pos2, pos3 = ja_pos[0], ja_pos[1], ja_pos[2]
        token_info = {'token_id': str(j),
                      'token': token,
                      'lemma': token.lemma_,
                      'pos': token.pos_,
                      'pos1': pos1,
                      'pos2': pos2,
                      'pos3': pos3, 
                      'dep': token.dep_, # 依存関係
                      'head': token.head, # 係り先
                      'head.id': token.head.i} # 係り先のインデックス
        #print(f"{token}\t{token.lemma_}\t{token.pos_}\t{token.tag_}\t{token.dep_}\t{token.head}\t{token.head.i}")
        sent_info[token] = token_info
    mt_info[str(i)] = sent_info
print(mt_info)

{'0': {'sent_id': '0', 'sent': '電子コモン・テクニカル・ドキュメント(eCTD)', 電子: {'token_id': '0', 'token': 電子, 'lemma': '電子', 'pos': 'NOUN', 'pos1': '名詞', 'pos2': '普通名詞', 'pos3': '一般', 'dep': 'compound', 'head': ドキュメント, 'head.id': 5}, コモン: {'token_id': '1', 'token': コモン, 'lemma': 'コモン', 'pos': 'NOUN', 'pos1': '名詞', 'pos2': '普通名詞', 'pos3': '一般', 'dep': 'compound', 'head': ドキュメント, 'head.id': 5}, ・: {'token_id': '2', 'token': ・, 'lemma': '・', 'pos': 'SYM', 'pos1': '補助記号', 'pos2': '一般', 'pos3': 'N', 'dep': 'compound', 'head': ドキュメント, 'head.id': 5}, テクニカル: {'token_id': '3', 'token': テクニカル, 'lemma': 'テクニカル', 'pos': 'PROPN', 'pos1': '形状詞', 'pos2': '一般', 'pos3': 'N', 'dep': 'compound', 'head': ドキュメント, 'head.id': 5}, ・: {'token_id': '4', 'token': ・, 'lemma': '・', 'pos': 'SYM', 'pos1': '補助記号', 'pos2': '一般', 'pos3': 'N', 'dep': 'compound', 'head': ドキュメント, 'head.id': 5}, ドキュメント: {'token_id': '5', 'token': ドキュメント, 'lemma': 'ドキュメント', 'pos': 'NOUN', 'pos1': '名詞', 'pos2': '普通名詞', 'pos3': '一般', 'dep': 'ROOT', 'head': ド